# PGE Budget Billing vs. Traditional

PGE has a new program, "Budget Billing":

"The Budget Billing program averages your energy costs over the previous 12 months to determine your monthly payment amount. If your actual energy costs significantly change, we adjust your monthly Budget Billing payment amount once every four months."

Questions for PGE:
+ What counts as "significant" change?

Questions for analysis:
+ Historically, would we have benefitted from this?
+ If so, what is the optimal month to start (given the 4-month update cycle)?

In [17]:
# starting after 1 full year, compute rolling 12-month average and compare it x4 to the sum of the next 4 months

# double check I pulled the full PGE data (I really want to start at December 2020)

## Function Definitions

In [18]:
import pandas as pd
import numpy as np
import re
import datetime

In [31]:
def process_csv(filename: str) -> tuple[pd.DataFrame, str]:
    """Converts a PGE csv file into a dataframe and its energy type name.

    Args:
        filename (str): The csv file path. Keep the PGE csv naming convention for proper parsing.

    Returns:
        df (pd.DataFrame): A dataframe of energy data. Dates are converted for proper merging with other PGE csv's.
        df_name (str): The energy type ('gas' or 'electric') as designated by PGE's csv naming convention.
    """

    df = pd.read_csv(filename, header=4, parse_dates=['START DATE', 'END DATE'])
    df['MID-DATE'] = df['START DATE'] + datetime.timedelta(days=5) # to accomodate start dates at the end of the month
    df['MONTH'] = df['MID-DATE'].dt.to_period('M')
    df_name = re.search(r'_([^_]*)_', filename).group(1)
    return df, df_name

def combine_and_process(df1: pd.DatFrame, df1_name: str, df2: pd.DataFrame, df2_name: str) -> pd.DataFrame:
    """Combines gas and electric data and computes rolling 12-month cost average.

    Args:
        df1 (pd.DataFrame): Gas or electric data parsed by the function process_csv.
        df1_name (str): The energy type of the first dataframe. This will be used as the suffix
            when the data is combined.
        df2 (pd.DataFrame): The other energy type data parsed by the function process_csv.
        df2_name (str): The energy type of the second dataframe. This will be used as the suffix
            when the data is combined.

    Returns:
        df (pd.DataFrame): A dataframe of the gas and electric data combined, with the rolling 12-month
            cost average computed.
    """

    df = pd.merge(left=df1, right=df2, left_on='MONTH', right_on='MONTH',
                suffixes=[f'_{df1_name}', f'_{df2_name}'])
    df['TOTAL_COST'] = df[f'COST_{df1_name}'].str.slice(start=1).astype(float)\
        + df[f'COST_{df2_name}'].str.slice(start=1).astype(float)
    df.sort_values(by='MONTH', ascending=True)
    df['1Y_ROLLING_AVG'] = df.rolling(12).mean().round(2)['TOTAL_COST']
    return df

TypeError: 'type' object is not subscriptable

In [32]:
def cost_comparison_quarterly_one_year(row_index: int) -> pd.DataFrame:
    """Computes the optimal quarterly PGE plan for a time span of one year.

    Args:
        row_index (int): The row index of the start month for the desired one year time span.
            The row refers to the dataframe generated by the combine_and_process function.

    Returns:
        df (pd.DataFrame): A 4-row dataframe where each row denotes a quarter of a year,
            showing which PGE plan is cheaper and by what dollar amount.
    """

    df = pd.DataFrame(columns=['start_month', 'end_month', 'cheaper_plan', 'cheaper_by_$'])
    start_row = row_index
    for i in range(4):
        start_month = total.iloc[start_row]['MONTH']
        end_month = total.iloc[start_row+3]['MONTH']
        budget_cost = total.iloc[start_row]['1Y_ROLLING_AVG']*4
        actual_cost = 0
        for i in range(4):
            actual_cost += total.iloc[start_row+i]['TOTAL_COST']
        cost_difference = actual_cost - budget_cost
        if cost_difference > 0:
            cheaper_plan = 'budget cost'
        if cost_difference < 0:
            cheaper_plan = 'actual cost'
            cost_difference *= -1
        if cost_difference == 0:
            cheaper_plan = 'no difference'
        df.loc[len(df.index)] = [start_month, end_month, cheaper_plan, cost_difference]
        start_row += 3
    return df

In [33]:
def cost_comparison_yearly(quarterly_df: pd.DataFrame) -> pd.DataFrame:
    """Computes the optimal yearly PGE plan.

    Args:
        quarterly_df (pd.DataFrame): A 4-row dataframe showing the optimal quarterly plan,
            as generated by the function cost_comparison_quarterly_one_year.

    Returns:
        df (pd.DataFrame): A 1-row dataframe showing the which PGE plan is cheaper for the
            entire year and by what dollar amount.
    """

    df = pd.DataFrame(columns=['start_month', 'end_month', 'cheaper_plan', 'cheaper_by_$'])
    start_month = quarterly_df['start_month'].min()
    end_month = quarterly_df['end_month'].max()
    quarterly_df['cheaper_by_$'].where(quarterly_df['cheaper_plan']=='actual cost', -quarterly_df['cheaper_by_$'], inplace=True)
    cost_difference = quarterly_df['cheaper_by_$'].sum()
    if cost_difference > 0:
        cheaper_plan = 'actual cost'
    if cost_difference < 0:
        cheaper_plan = 'budget cost'
        cost_difference *= -1
    if cost_difference == 0:
        cheaper_plan = 'no difference'
    df.loc[len(df.index)] = [start_month, end_month, cheaper_plan, cost_difference]
    return df

In [34]:
def total_analysis(combined_df: pd.DataFrame) -> pd.DataFrame:
    """Computes the optimal PGE plan and start date for all input data.

    The idea is that if the PGE Budget Billing plan maximized savings for a given year 
    starting in March of that year, for example, then one could assume that it would be
    best to switch to the Budget Billing plan the next possible March.

    Args:
        combined_df (pd.DataFrame): A dataframe of PGE gas and electric data, as parsed by
            the function combine_and_process.

    Returns:
        df (pd.DataFrame): A 1-row dataframe which is the maximum savings possible given
            the input data.
    """

    df = pd.DataFrame(columns=['start_month', 'end_month', 'cheaper_plan', 'cheaper_by_$'])
    start_month_index = np.isnan(combined_df['1Y_ROLLING_AVG']).argmin(axis=0) # first non-null rolling average
    for i in range(start_month_index, len(combined_df.index)-12):
        df = df.append(cost_comparison_yearly(cost_comparison_quarterly_one_year(i)), ignore_index=True)
    return df

## Data Analysis

In [35]:
gas, gas_name = process_csv('data/pge_gas_billing_data_4321483928_2021-04-02_to_2024-03-01.csv')
electric, electric_name = process_csv('data/pge_electric_billing_data_4323839271_2021-04-01_to_2024-02-29.csv')
total = combine_and_process(gas, gas_name, electric, electric_name)

In [36]:
start_month_index = np.isnan(total['1Y_ROLLING_AVG']).argmin(axis=0) # first non-null rolling average

In [37]:
cost_comparison_quarterly_one_year(start_month_index)

,start_month,end_month,cheaper_plan,cheaper_by_$
0,2022-03,2022-06,actual cost,37.68
1,2022-06,2022-09,actual cost,108.15
2,2022-09,2022-12,actual cost,39.87
3,2022-12,2023-03,budget cost,164.32


In [38]:
cost_comparison_yearly(cost_comparison_quarterly_one_year(start_month_index))

,start_month,end_month,cheaper_plan,cheaper_by_$
0,2022-03,2023-03,actual cost,21.38


In [39]:
total_analysis(total)

,start_month,end_month,cheaper_plan,cheaper_by_$
0,2022-03,2023-03,actual cost,21.38
1,2022-04,2023-04,budget cost,7.59
2,2022-05,2023-05,budget cost,13.83
3,2022-06,2023-06,actual cost,27.53
4,2022-07,2023-07,actual cost,6.34
5,2022-08,2023-08,budget cost,13.29
6,2022-09,2023-09,actual cost,17.86
7,2022-10,2023-10,budget cost,25.32
8,2022-11,2023-11,budget cost,17.62
9,2022-12,2023-12,actual cost,18.34


In [40]:
total_analysis_df = total_analysis(total)
total_analysis_df.iloc[total_analysis_df['cheaper_by_$'].idxmax()].to_frame().T

,start_month,end_month,cheaper_plan,cheaper_by_$
11,2023-02,2024-02,actual cost,56.81


In [41]:
round(total_analysis_df[total_analysis_df['cheaper_plan'].isin(['actual cost'])]['cheaper_by_$'].mean(), 2)

24.71

In [42]:
round(total_analysis_df[total_analysis_df['cheaper_plan'].isin(['budget cost'])]['cheaper_by_$'].mean(), 2)

13.68

In [ ]:
# plots
from matplotlib import pyplot as plt

In [ ]:
plt.plot(total.index, total['TOTAL COST'])

In [ ]:
total.dtypes

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.bar(x=total['MONTH'].dt.to_timestamp(), height=total['TOTAL COST'], width=25)
ax.xaxis_date()
ax.tick_params(axis='x', rotation=60)
ax.set_xticks(total['MONTH'].dt.to_timestamp())
ax
# color by year